In [1]:
# !pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import logging
import multiprocessing
from datetime import datetime, timedelta

from src import modules as f
%load_ext autoreload
%autoreload 2

In [3]:
stock_list = f.download_interval_process('1h', 1)

In [4]:
stock_list

{'AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'ADBE',
 'AEP',
 'AJG',
 'AMGN',
 'AMZN',
 'APH',
 'ATO',
 'AVGO',
 'AXP',
 'BAC',
 'BRK-B',
 'BSX',
 'CEG',
 'CL',
 'CMCSA',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVX',
 'D',
 'DASH',
 'DECK',
 'DIS',
 'DKNG',
 'DUK',
 'EOG',
 'EXC',
 'EXE',
 'FCNCA',
 'FWONK',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'IBM',
 'ISRG',
 'JNJ',
 'JPM',
 'KMI',
 'KO',
 'LLY',
 'LNG',
 'MA',
 'MDLZ',
 'META',
 'MO',
 'MRK',
 'MSFT',
 'MSI',
 'NEE',
 'NFLX',
 'NRG',
 'NVDA',
 'OKE',
 'ORCL',
 'PEP',
 'PG',
 'PGR',
 'PLTR',
 'PM',
 'PSX',
 'RCL',
 'SLB',
 'SO',
 'SPGI',
 'SRE',
 'SW',
 'T',
 'TDG',
 'TGT',
 'TMO',
 'TMUS',
 'TPR',
 'TSLA',
 'TTWO',
 'UNH',
 'V',
 'VST',
 'VZ',
 'WELL',
 'WFC',
 'WMB',
 'WMT',
 'WSM',
 'XEL',
 'XOM',
 '^VIX'}

In [7]:
stock_list = f.make_table_features_process('1h', 8)

In [8]:
f.make_master_table(stock_list, '1h')

Saved combined dataframe to: ./data_transformed/all_1h_model_df.pkl


In [9]:
f.xg_boost_model(interval='1h')

/home/oem/Documents/github/stock_production_agg/stkenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [00:22:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.47      0.53      0.50     45982
           1       0.46      0.44      0.45     45995
           2       0.47      0.42      0.45     45989

    accuracy                           0.47    137966
   macro avg       0.47      0.47      0.47    137966
weighted avg       0.47      0.47      0.47    137966


Feature Importances (Descending):
                    feature  importance
57       cat__hour_of_day_9    0.188551
63      cat__hour_of_day_15    0.094397
64      cat__hour_of_day_16    0.063112
65      cat__hour_of_day_17    0.060009
53       cat__hour_of_day_5    0.032134
..                      ...         ...
69  cat__candle_cluster_1.0    0.001854
73  cat__candle_cluster_5.0    0.001742
71  cat__candle_cluster_3.0    0.001487
3    cat__fast_sma_signal_1    0.000000
1    cat__slow_sma_signal_1    0.000000

[76 rows x 2 columns]

Model saved at: ./models/xgboost_1h_model.pkl
